## 9: Cleanup Resources

This notebook helps you clean up all AWS resources created during 1-8 to avoid unnecessary charges.

**⚠️ WARNING:** This will delete all resources created in the workshop. Make sure you've saved any important data or configurations before proceeding.

## Resources to Clean Up:

- **1**: AgentCore Memory
- **2**: Cognito User Pool, IAM Execution Role
- **3**: AgentCore Gateway, Gateway Targets, Gateway IAM Role
- **4**: Memory events (already cleaned by Memory deletion)
- **5**: AgentCore Runtime, ECR Repository, CodeBuild Project
- **6**: CloudWatch Logs (optional)
- **7**: Policy Engine, Cedar Policies
- **8**: SSM Parameters, Streamlit resources
- **DynamoDB**: Table records created by Lambda functions

**Note:** Knowledge Base and Lambda functions from CloudFormation are NOT deleted by this notebook.

### Step 1: Initialize and Load Configurations

In [ ]:
import boto3
import json
import os
from botocore.exceptions import ClientError

# Initialize AWS clients
session = boto3.Session()
region = session.region_name or 'us-west-2'
sts = session.client('sts')
identity = sts.get_caller_identity()
account_id = identity['Account']

print(f"🔍 Cleanup Configuration:")
print(f"   Account ID: {account_id}")
print(f"   Region: {region}")
print(f"\n⚠️  This will delete resources in the above account/region.")

# Load configuration files
configs = {}
config_files = ['cognito_config.json', 'gateway_config.json', 'memory_config.json', 'runtime_config.json']

for config_file in config_files:
    if os.path.exists(config_file):
        with open(config_file, 'r') as f:
            configs[config_file] = json.load(f)
        print(f"✅ Loaded {config_file}")
    else:
        print(f"⚠️  {config_file} not found - skipping related resources")

print(f"\n📋 Found {len(configs)} configuration files")

### Step 2: Delete AgentCore Runtime (Agent)

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 2: Deleting AgentCore Runtime (Agent)")
print("="*80)

if 'runtime_config.json' in configs:
    try:
        agent_arn = configs['runtime_config.json'].get('agent_arn')
        if agent_arn:
            print(f"\n📍 Found agent: {agent_arn}")
            
            # Extract agent ID from ARN
            agent_id = agent_arn.split('/')[-1]
            
            # Delete the agent using AWS SDK
            print("   Deleting agent runtime...")
            try:
                agentcore_client = boto3.client('bedrock-agentcore-control', region_name=region)
                agentcore_client.delete_agent_runtime(agentRuntimeId=agent_id)
                print("   ✅ Agent runtime deleted")
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print("   ℹ️  Agent already deleted or not found")
                else:
                    print(f"   ⚠️  Error deleting agent: {e}")
            except Exception as e:
                if "ResourceNotFoundException" in str(e) or "not found" in str(e).lower():
                    print("   ℹ️  Agent already deleted or not found")
                else:
                    print(f"   ⚠️  Error deleting agent: {e}")
        else:
            print("   ⚠️  No agent ARN found in config")
    except Exception as e:
        print(f"   ❌ Error: {e}")
else:
    print("   ℹ️  No runtime config found - skipping")

print("\n✅ Runtime cleanup complete")

### Step 3: Delete Gateway Policy Engine

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 3: Deleting Policy Engine")
print("="*80)

# Policy engines are automatically deleted when the gateway is deleted
# No manual action required - just verify if one exists

try:
    if 'gateway_config.json' in configs:
        gateway_id = configs['gateway_config.json'].get('id')
        
        if gateway_id:
            print("\n📍 Checking if policy engine is attached to gateway...")
            
            try:
                agentcore_client = boto3.client('bedrock-agentcore-control', region_name=region)
                
                # Get gateway details to check if policy engine is attached
                gateway_response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
                policy_config = gateway_response.get('policyEngineConfiguration')
                
                if policy_config and policy_config.get('arn'):
                    policy_engine_arn = policy_config['arn']
                    policy_engine_id = policy_engine_arn.split('/')[-1]
                    print(f"   Found attached policy engine: {policy_engine_id}")
                    print("   ℹ️  Policy engine will be automatically deleted when gateway is deleted")
                else:
                    print("   ℹ️  No policy engine attached to gateway")
                    
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print("   ℹ️  Gateway not found - policy engine already cleaned up")
                else:
                    print(f"   ℹ️  Could not check policy engine: {e}")
            except Exception as e:
                print(f"   ℹ️  Could not check policy engine: {e}")
    else:
        print("\n   ℹ️  No gateway config found")
    
    print("\n   💡 Policy engines are automatically deleted when the gateway is deleted in Step 4")
        
except Exception as e:
    print(f"   ⚠️  Error: {e}")

print("\n✅ Policy engine cleanup complete")

### Step 4: Delete AgentCore Gateway

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 4: Deleting AgentCore Gateway")
print("="*80)

if 'gateway_config.json' in configs:
    try:
        gateway_client = boto3.client('bedrock-agentcore-control', region_name=region)
        gateway_id = configs['gateway_config.json'].get('id')
        
        if gateway_id:
            print(f"\n📍 Found gateway: {gateway_id}")
            
            # Check if gateway exists
            try:
                gateway_client.get_gateway(gatewayIdentifier=gateway_id)
                
                # First, delete all gateway targets
                print("   Listing gateway targets...")
                try:
                    targets_response = gateway_client.list_gateway_targets(gatewayIdentifier=gateway_id)
                    targets = targets_response.get('items', [])
                    
                    if targets:
                        print(f"   Found {len(targets)} target(s) to delete")
                        for target in targets:
                            target_id = target.get('targetId')
                            target_name = target.get('name', 'Unknown')
                            try:
                                print(f"   Deleting target: {target_name} ({target_id})")
                                gateway_client.delete_gateway_target(
                                    gatewayIdentifier=gateway_id,
                                    targetId=target_id
                                )
                                print(f"   ✅ Target deleted: {target_name}")
                            except Exception as e:
                                print(f"   ⚠️  Error deleting target {target_name}: {e}")
                        
                        # Wait for target deletions to propagate
                        import time
                        print("   Waiting for target deletions to propagate...")
                        time.sleep(5)
                    else:
                        print("   ℹ️  No targets found")
                except Exception as e:
                    print(f"   ⚠️  Error listing targets: {e}")
                
                # Now delete the gateway
                print("   Deleting gateway...")
                gateway_client.delete_gateway(gatewayIdentifier=gateway_id)
                print("   ✅ Gateway deleted")
                
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print("   ℹ️  Gateway already deleted or not found")
                else:
                    raise
        else:
            print("   ⚠️  No gateway ID found in config")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")
else:
    print("   ℹ️  No gateway config found - skipping")

print("\n✅ Gateway cleanup complete")

### Step 5: Delete Gateway IAM Role

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 5: Deleting Gateway IAM Role")
print("="*80)

try:
    iam_client = boto3.client('iam')
    role_name = "RefundManagementGatewayExecutionRole"
    
    print(f"\n📍 Checking for role: {role_name}")
    
    try:
        # Check if role exists
        iam_client.get_role(RoleName=role_name)
        
        # Delete inline policies first
        print("   Deleting inline policies...")
        try:
            policies = iam_client.list_role_policies(RoleName=role_name)
            for policy_name in policies.get('PolicyNames', []):
                iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
                print(f"   ✅ Deleted policy: {policy_name}")
        except Exception as e:
            print(f"   ⚠️  Error deleting policies: {e}")
        
        # Delete the role
        print("   Deleting IAM role...")
        iam_client.delete_role(RoleName=role_name)
        print("   ✅ IAM role deleted")
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print("   ℹ️  Role already deleted or not found")
        else:
            raise
            
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n✅ Gateway IAM role cleanup complete")

### Step 6: Delete AgentCore Memory

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 6: Deleting AgentCore Memory")
print("="*80)

if 'memory_config.json' in configs:
    try:
        from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
        
        memory_id = configs['memory_config.json'].get('memory_id')
        memory_name = configs['memory_config.json'].get('memory_name', 'ReturnRefundAssisantMemory')
        
        if memory_id:
            print(f"\n📍 Found memory: {memory_id}")
            
            memory_manager = MemoryManager(region_name=region)
            
            # Check if memory exists
            try:
                print("   Deleting memory...")
                memory_manager.delete_memory(memory_id=memory_id)
                print("   ✅ Memory deleted")
            except Exception as e:
                if "ResourceNotFoundException" in str(e) or "not found" in str(e).lower():
                    print("   ℹ️  Memory already deleted or not found")
                else:
                    print(f"   ⚠️  Error deleting memory: {e}")
        else:
            print("   ⚠️  No memory ID found in config")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")
else:
    print("   ℹ️  No memory config found - skipping")

print("\n✅ Memory cleanup complete")

### Step 7: Delete Cognito User Pool

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 7: Deleting Cognito User Pool")
print("="*80)

if 'cognito_config.json' in configs:
    try:
        cognito_client = boto3.client('cognito-idp', region_name=region)
        pool_id = configs['cognito_config.json'].get('pool_id')
        
        if pool_id:
            print(f"\n📍 Found user pool: {pool_id}")
            
            # Check if pool exists
            try:
                cognito_client.describe_user_pool(UserPoolId=pool_id)
                
                # Delete user pool domain first (if exists)
                try:
                    print("   Checking for user pool domain...")
                    pool_desc = cognito_client.describe_user_pool(UserPoolId=pool_id)
                    domain = pool_desc.get('UserPool', {}).get('Domain')
                    if domain:
                        print(f"   Deleting domain: {domain}")
                        cognito_client.delete_user_pool_domain(Domain=domain, UserPoolId=pool_id)
                        print("   ✅ Domain deleted")
                except Exception as e:
                    print(f"   ℹ️  No domain to delete or error: {e}")
                
                # Delete user pool
                print("   Deleting user pool...")
                cognito_client.delete_user_pool(UserPoolId=pool_id)
                print("   ✅ User pool deleted")
                
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print("   ℹ️  User pool already deleted or not found")
                else:
                    raise
        else:
            print("   ⚠️  No pool ID found in config")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")
else:
    print("   ℹ️  No Cognito config found - skipping")

print("\n✅ Cognito cleanup complete")

### Step 8: Delete Runtime Execution IAM Role

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 8: Deleting Runtime Execution IAM Role")
print("="*80)

if 'cognito_config.json' in configs:
    try:
        iam_client = boto3.client('iam')
        execution_role_arn = configs['cognito_config.json'].get('execution_role')
        
        if execution_role_arn:
            # Extract role name from ARN
            role_name = execution_role_arn.split('/')[-1]
            print(f"\n📍 Found execution role: {role_name}")
            
            try:
                # Check if role exists
                iam_client.get_role(RoleName=role_name)
                
                # Delete inline policies
                print("   Deleting inline policies...")
                try:
                    policies = iam_client.list_role_policies(RoleName=role_name)
                    for policy_name in policies.get('PolicyNames', []):
                        iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
                        print(f"   ✅ Deleted policy: {policy_name}")
                except Exception as e:
                    print(f"   ⚠️  Error deleting policies: {e}")
                
                # Detach managed policies
                print("   Detaching managed policies...")
                try:
                    attached = iam_client.list_attached_role_policies(RoleName=role_name)
                    for policy in attached.get('AttachedPolicies', []):
                        iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy['PolicyArn'])
                        print(f"   ✅ Detached policy: {policy['PolicyName']}")
                except Exception as e:
                    print(f"   ⚠️  Error detaching policies: {e}")
                
                # Delete the role
                print("   Deleting IAM role...")
                iam_client.delete_role(RoleName=role_name)
                print("   ✅ IAM role deleted")
                
            except ClientError as e:
                if e.response['Error']['Code'] == 'NoSuchEntity':
                    print("   ℹ️  Role already deleted or not found")
                else:
                    raise
        else:
            print("   ⚠️  No execution role ARN found in config")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")
else:
    print("   ℹ️  No Cognito config found - skipping")

print("\n✅ Execution role cleanup complete")

### Step 9: Delete SSM Parameters

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 9: Deleting SSM Parameters")
print("="*80)

try:
    ssm_client = boto3.client('ssm', region_name=region)
    
    # List of SSM parameters to delete
    ssm_parameters = [
        '/app/returnsrefunds/agentcore/runtime_arn'
    ]
    
    for param_name in ssm_parameters:
        try:
            print(f"\n📍 Checking parameter: {param_name}")
            ssm_client.get_parameter(Name=param_name)
            
            # Delete parameter
            ssm_client.delete_parameter(Name=param_name)
            print(f"   ✅ Parameter deleted")
            
        except ClientError as e:
            if e.response['Error']['Code'] == 'ParameterNotFound':
                print(f"   ℹ️  Parameter not found or already deleted")
            else:
                print(f"   ⚠️  Error: {e}")
                
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n✅ SSM parameters cleanup complete")

### Step 10: Delete ECR Repository

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 10: Deleting ECR Repository")
print("="*80)

try:
    ecr_client = boto3.client('ecr', region_name=region)
    repo_name = "bedrock-agentcore-returns_refunds_agent"
    
    print(f"\n📍 Checking for repository: {repo_name}")
    
    try:
        # Check if repository exists
        ecr_client.describe_repositories(repositoryNames=[repo_name])
        
        # Delete repository (force=True deletes even if it contains images)
        print("   Deleting ECR repository...")
        ecr_client.delete_repository(repositoryName=repo_name, force=True)
        print("   ✅ ECR repository deleted")
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'RepositoryNotFoundException':
            print("   ℹ️  Repository already deleted or not found")
        else:
            raise
            
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n✅ ECR repository cleanup complete")

### Step 11: Delete CodeBuild Project

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 11: Deleting CodeBuild Project")
print("="*80)

try:
    codebuild_client = boto3.client('codebuild', region_name=region)
    project_name = "bedrock-agentcore-returns_refunds_agent-builder"
    
    print(f"\n📍 Checking for project: {project_name}")
    
    try:
        # Check if project exists
        codebuild_client.batch_get_projects(names=[project_name])
        
        # Delete project
        print("   Deleting CodeBuild project...")
        codebuild_client.delete_project(name=project_name)
        print("   ✅ CodeBuild project deleted")
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print("   ℹ️  Project already deleted or not found")
        else:
            raise
            
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n✅ CodeBuild project cleanup complete")

### Step 12: Delete CodeBuild IAM Role

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 12: Deleting CodeBuild IAM Role")
print("="*80)

try:
    iam_client = boto3.client('iam')
    
    # List roles that match the CodeBuild pattern
    role_prefix = f"AmazonBedrockAgentCoreSDKCodeBuild-{region}"
    
    print(f"\n📍 Searching for CodeBuild roles with prefix: {role_prefix}")
    
    try:
        # List all roles and filter
        paginator = iam_client.get_paginator('list_roles')
        for page in paginator.paginate():
            for role in page['Roles']:
                if role['RoleName'].startswith(role_prefix):
                    role_name = role['RoleName']
                    print(f"\n   Found role: {role_name}")
                    
                    try:
                        # Delete inline policies
                        policies = iam_client.list_role_policies(RoleName=role_name)
                        for policy_name in policies.get('PolicyNames', []):
                            iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
                            print(f"      ✅ Deleted inline policy: {policy_name}")
                        
                        # Detach managed policies
                        attached = iam_client.list_attached_role_policies(RoleName=role_name)
                        for policy in attached.get('AttachedPolicies', []):
                            iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy['PolicyArn'])
                            print(f"      ✅ Detached managed policy: {policy['PolicyName']}")
                        
                        # Delete the role
                        iam_client.delete_role(RoleName=role_name)
                        print(f"      ✅ Role deleted")
                        
                    except Exception as e:
                        print(f"      ⚠️  Error deleting role: {e}")
        
        print("\n   ℹ️  CodeBuild role search complete")
        
    except Exception as e:
        print(f"   ⚠️  Error searching roles: {e}")
        
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n✅ CodeBuild IAM role cleanup complete")

### Step 13: Clean Up Local Configuration Files

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 13: Cleaning Up Local Configuration Files")
print("="*80)

local_files = [
    'cognito_config.json',
    'gateway_config.json',
    'memory_config.json',
    'runtime_config.json',
    'kb_config.json',
    '.bedrock_agentcore.yaml',
    'agent_runtime.py',
    'Dockerfile'
]

print("\n📍 Removing local configuration files...\n")

for file_name in local_files:
    if os.path.exists(file_name):
        try:
            os.remove(file_name)
            print(f"   ✅ Deleted: {file_name}")
        except Exception as e:
            print(f"   ⚠️  Error deleting {file_name}: {e}")
    else:
        print(f"   ℹ️  Not found: {file_name}")

print("\n✅ Local files cleanup complete")

### Step 14: Delete DynamoDB Table Records

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 14: Deleting DynamoDB Table Records")
print("="*80)

try:
    dynamodb = boto3.resource('dynamodb', region_name=region)
    dynamodb_client = boto3.client('dynamodb', region_name=region)
    
    # Search for DynamoDB tables related to the workshop
    # Common patterns: refund, return, workshop
    table_patterns = ['refund', 'return', 'workshop', 'agentcore']
    
    print("\n📍 Searching for DynamoDB tables...")
    
    try:
        # List all tables
        response = dynamodb_client.list_tables()
        all_tables = response.get('TableNames', [])
        
        # Filter tables that match our patterns (case-insensitive)
        matching_tables = []
        for table_name in all_tables:
            for pattern in table_patterns:
                if pattern.lower() in table_name.lower():
                    matching_tables.append(table_name)
                    break
        
        if matching_tables:
            print(f"   Found {len(matching_tables)} table(s) matching workshop patterns:\n")
            for table_name in matching_tables:
                print(f"   📋 Table: {table_name}")
            
            print("\n⚠️  Deleting ALL records from the tables above...\n")
            
            # Delete records from all matching tables
            for table_name in matching_tables:
                print(f"   Processing table: {table_name}")
                table = dynamodb.Table(table_name)
                
                try:
                    # Get table key schema to know which attributes are keys
                    table_info = dynamodb_client.describe_table(TableName=table_name)
                    key_schema = table_info['Table']['KeySchema']
                    key_names = [key['AttributeName'] for key in key_schema]
                    
                    print(f"      Key attributes: {', '.join(key_names)}")
                    
                    # Scan and delete all items
                    scan_kwargs = {}
                    deleted_count = 0
                    
                    while True:
                        response = table.scan(**scan_kwargs)
                        items = response.get('Items', [])
                        
                        if not items:
                            break
                        
                        # Delete items in batches
                        with table.batch_writer() as batch:
                            for item in items:
                                # Extract only the key attributes for deletion
                                key = {k: item[k] for k in key_names if k in item}
                                batch.delete_item(Key=key)
                                deleted_count += 1
                        
                        # Check if there are more items to scan
                        if 'LastEvaluatedKey' not in response:
                            break
                        scan_kwargs['ExclusiveStartKey'] = response['LastEvaluatedKey']
                    
                    print(f"      ✅ Deleted {deleted_count} record(s) from {table_name}\n")
                    
                except Exception as e:
                    print(f"      ⚠️  Error deleting records from {table_name}: {e}\n")
            
        else:
            print("   ℹ️  No matching DynamoDB tables found")
            print("   💡 If you have a specific table name, add it to the table_patterns list")
        
    except Exception as e:
        print(f"   ⚠️  Error listing tables: {e}")
        
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n✅ DynamoDB records cleanup complete")

### Step 15: Optional - Delete CloudWatch Log Groups

In [ ]:
print("\n" + "="*80)
print("🗑️  STEP 15: Deleting CloudWatch Log Groups (OPTIONAL)")
print("="*80)
print("\n⚠️  This will delete all agent logs. Uncomment the code below to proceed.\n")

try:
    logs_client = boto3.client('logs', region_name=region)
    
    # Search for agent log groups
    log_group_prefix = '/aws/bedrock-agentcore/runtimes/returns_refunds_agent'
    
    print(f"📍 Searching for log groups with prefix: {log_group_prefix}")
    
    try:
        response = logs_client.describe_log_groups(logGroupNamePrefix=log_group_prefix)
        
        for log_group in response.get('logGroups', []):
            log_group_name = log_group['logGroupName']
            print(f"\n   Found log group: {log_group_name}")
            
            try:
                logs_client.delete_log_group(logGroupName=log_group_name)
                print(f"   ✅ Log group deleted")
            except Exception as e:
                print(f"   ⚠️  Error deleting log group: {e}")
        
        if not response.get('logGroups'):
            print("   ℹ️  No log groups found")
            
    except Exception as e:
        print(f"   ⚠️  Error searching log groups: {e}")
        
except Exception as e:
    print(f"   ❌ Error: {e}")

print("   ℹ️  CloudWatch logs cleanup skipped (commented out)")
print("\n✅ CloudWatch logs step complete")

### Summary

In [ ]:
print("\n" + "="*80)
print("✅ CLEANUP COMPLETE")
print("="*80)

print("\n📋 Resources Cleaned Up:")
print("   ✅ AgentCore Runtime (Agent)")
print("   ✅ Policy Engine")
print("   ✅ AgentCore Gateway")
print("   ✅ Gateway IAM Role")
print("   ✅ AgentCore Memory")
print("   ✅ Cognito User Pool")
print("   ✅ Runtime Execution IAM Role")
print("   ✅ SSM Parameters")
print("   ✅ ECR Repository")
print("   ✅ CodeBuild Project")
print("   ✅ CodeBuild IAM Role")
print("   ✅ Local Configuration Files")
print("   ✅ DynamoDB Table Records")
print("   ⏭️  CloudWatch Logs (skipped - optional)")

print("\n📝 Resources NOT deleted (manual cleanup required):")
print("   • Knowledge Base (created via CloudFormation)")
print("   • Lambda functions (CreateRefundRequest, ListReturnRequest, ApproveReturnRequest)")
print("   • CloudFormation stacks")

print("\n💡 To delete the Knowledge Base and Lambda functions:")
print("   1. Go to AWS CloudFormation console")
print("   2. Delete the stack that created these resources")

print("\n🎉 Workshop cleanup complete! Thank you for using Amazon Bedrock AgentCore.")